In [76]:
LIMIT = 2000
articles_array = []

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [ ]:
count  =  1

  # Iterate through each news company# Itera 
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data. RSS (Rich Site Summary; originally RDF Site Summary; often called Really Simple Syndication) is a type of
    # web feed which allows users to access updates to online content in a standardized, computer-readable format
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                article['authors'] = content.authors
                article['top_image'] =  content.top_image
                article['movies'] = content.movies
                newsPaper['articles'].append(article)
                articles_array.append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.

            article = {}
            article['title'] = content.title
            article['authors'] = content.authors
            article['text'] = content.text
            article['top_image'] =  content.top_image
            article['movies'] = content.movies
            article['link'] = content.url
            article['published'] = content.publish_date
            newsPaper['articles'].append(article)
            articles_array.append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            #noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

Building site for  realagriculture
1 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/7-things-we-learned-at-ag-in-motion/
2 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/japan-resumes-purchases-of-canadian-wheat/
3 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/ontario-sheep-farmers-offering-master-shepherds-course-for-fall-2018/
4 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/ontario-joins-saskatchewan-in-legal-case-against-federal-carbon-tax/
5 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/macaulay-retains-ag-post-in-pre-election-cabinet-shuffle/
6 articles downloaded from realagriculture  using newspaper, url:  https://www.realagriculture.com/2018/07/clean-seed-signs-deal-to-buy-harvest-internati

13 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/gm-wheat-policy-called-irresponsible/
14 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/how-to-face-future-land-value-changes/
15 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/feds-must-support-farmers-if-trade-war-turns-ugly/
16 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/ag-canada-raises-canola-carryout-forecast/
17 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/trade-uncertainty-unnerves-the-markets/
18 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/red-lentil-prices-could-rise/
19 articles downloaded from producer  using newspaper, url:  https://www.producer.com/2018/07/firm-steady-prices-expected-for-flax-this-year/
20 articles downloaded from producer  using newspaper

In [23]:
data

{'newspapers': {'agcanada': {'articles': [{'authors': ['Farm Business Communications'],
     'link': 'https://www.agcanada.com/daily/worlds-affection-for-canada-will-help-in-trade-diversification-carr',
     'movies': [],
     'published': None,
     'text': 'Ottawa | Reuters — There is “an awful lot of affection for Canada” around the world that will help the government’s push to diversify exports away from the United States, the country’s new trade minister said on Thursday, as ties with Washington become more strained.\n\nWinnipeg MP Jim Carr, previously the minister for natural resources, received the trade portfolio in a cabinet reshuffle on Wednesday, when Prime Minister Justin Trudeau said finding new export markets was a priority.\n\nCanada sends 75 per cent of its goods exports to the U.S., which imposed tariffs on Canadian steel and aluminum at the end of May and is now mulling punitive measures against autos.\n\n“There is goodwill towards Canada, there is a willingness to he

In [26]:
  #Finally it saves the articles as a CSV-file.
try:
    f = csv.writer(open('Scraped_data_news_output12.csv', 'w', encoding='utf-8'))
    f.writerow(['Title', 'Authors','Text','Image','Videos','Link','Published_Date'])
    #print(article)
    for artist_name in articles_array:
            title = artist_name['title']
            authors=artist_name['authors']
            text=artist_name['text']
            image=artist_name['top_image']
            video=artist_name['movies']
            link=artist_name['link']
            publish_date=artist_name['published']
            # Add each artist’s name and associated link to a row
            f.writerow([title, authors, text, image, video, link, publish_date])
            
except Exception as e: print(e)

In [ ]:
in_fnam='Scraped_data_news_output12.csv'
out_fnam='out.csv'
input = open(in_fnam, 'rb')
output = open(out_fnam, 'wb')
writer = csv.writer(output)
for row in csv.reader(input):
    if row:
        writer.writerow(row)
input.close()
output.close()